In [ ]:
import pandas as pd
import numpy as np
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time
import random
import re
from collections import Counter

from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from wordcloud import WordCloud

print("✅ All libraires Imported")

In [ ]:
class TweetGenerator:
    """Generate sample teets"""
    
    def __init__(self):
        self.positive_tweets = [
            "I love this new product! Amazing quality! 😍",
            "Having a great day with friends! #blessed",
            "Just got promoted at work! So excited! 🎉",
            "Beautiful sunset today! Nature is amazing 🌅",
            "Great customer service experience! Highly recommend!",
            "Finally finished my project! Feeling accomplished ✨",
            "Best movie I've watched this year! Must see! 🍿",
            "Delicious food at this restaurant! Coming back soon!",
            "Love spending time with family! ❤️",
            "Successfully completed my workout! Feeling strong 💪"
        ]
        
        self.negative_tweets = [
            "Terrible customer service! Very disappointed 😠",
            "This product broke after one day! Waste of money!",
            "Stuck in traffic for 2 hours! Worst day ever!",
            "Can't believe how rude people can be! 😤",
            "My order was completely wrong! Very frustrated!",
            "Website is so slow and buggy! Fix this please!",
            "Overpriced and poor quality! Not worth it!",
            "Rain ruined my outdoor plans! So annoying ☔",
            "Long wait times everywhere! What's happening?",
            "Technology failing me when I need it most! 📱💔"
        ]
        
        self.neutral_tweets = [
            "Just finished reading a book about data science.",
            "Weather forecast shows rain tomorrow.",
            "The meeting is scheduled for 3 PM.",
            "Coffee shop on main street opens at 7 AM.",
            "Train arrives at platform 2 in 5 minutes.",
            "New policy update will be effective next month.",
            "Concert tickets go on sale this Friday.",
            "Library hours: Monday to Friday 9 AM to 8 PM.",
            "Parking meter accepts both coins and cards.",
            "Store inventory will be updated weekly."
        ]
        
        self.topics = ["#technology", "#food", "#travel", "#work", "#family", "#health", "#sports", "#entertainment"]
        
    def generate_tweet(self):
        sentiment_type = random.choices(
            ['positive', 'negative', 'neutral'],
            weights = [0.4, 0.3, 0.3]
        )[0]
        
        if sentiment_type == 'positive':
            text = random.choice(self.positive_tweets)
        elif sentiment_type == 'negative':
            text = random.choice(self.negative_tweets)
        else:
            text = random.choice(self.neutral_tweets)
            
        if random.random() > 0.5:
            text += " " + random.choice(self.topics)
            
        return {
            'text': text,
            'timestamp': datetime.now() - timedelta(seconds=random.randint(0, 3600)),
            'user': f"user_{random.randint(1000, 9999)}",
            'likes': random.randint(0, 100),
            'retweets': random.randint(0, 50)
        }
        
    def generate_batach_tweets(self, count=10):
        return [self.generate_tweet() for _ in range(count)]    

In [ ]:
class SentimentAnalyzer:
    """Multi-modal sentiment analysis"""
    
    def __init__(self) -> None:
        self.vader_analyzer = SentimentIntensityAnalyzer()
        
    def clean_text(self, text):
        # Removing URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        # Removing mentions and all also
        text = re.sub(r'@\w+|#', '', text)
        text = ' '.join(text.split())
        
        return text.strip()
    
    def analyze_textblob(self, text):
        clean_text = self.clean_text(text)
        blob = TextBlob(clean_text)
        
        polarity = blob.sentiment.polarity
        subjectivity = blob.sentiment.subjectivity
        
        if polarity > 0.1:
            sentiment = 'Positive'
        elif polarity < -0.1:
            sentiment = 'Negative'
        else:
            sentiment = 'Neutral'
            
        return {
            'sentiment': sentiment,
            'polarity': polarity,
            'subjectivity': subjectivity,
            'confidence': abs(polarity)
        }
        
    def analyze_vader(self, text):
        clean_text = self.clean_text(text)
        scores = self.vader_analyzer.polarity_scores(clean_text)
        
        if scores['compound'] >= 0.05:
            sentiment = 'Positive'
        elif scores['compound'] <= -0.05:
            sentiment = 'Negative'
        else:
            sentiment = 'Neutral'
            
        return {
            'sentiment': sentiment,
            'positive': scores['pos'],
            'neutral': scores['neu'],
            'negative': scores['neg'],
            'compound': scores['compound']
        }
        
    def analyze_combined(self, text):
        textblob_result = self.analyze_textblob(text)
        vader_result = self.analyze_vader(text)
        
        if textblob_result['sentiment'] == vader_result['sentiment']:
            final_sentiment = textblob_result['sentiment']
            confidence = (abs(textblob_result['polarity']) + abs(vader_result['compound'])) / 2
        else:
            if abs(vader_result['compound']) > abs(textblob_result['polarity']):
                final_sentiment = vader_result['sentiment']
                confidence = abs(vader_result['compound'])
            else:
                final_sentiment = textblob_result['sentiment']
                confidence = abs(textblob_result['polarity'])
        
        return {
            'sentiment': final_sentiment,
            'confidence': confidence,
            'textblob': textblob_result,
            'vader': vader_result
        }
        
    def analyze_batch(self, texts):
        results = []
        
        for text in texts:
            result = self.analyze_combined(text)
            results.append(result)
    
        return results
        